# Import Library

In [2]:
import sqlite3
from time import sleep
import numpy as np
import requests
import json
import pandas as pd
import warnings
import tweepy

warnings.simplefilter(action='ignore', category=FutureWarning)

# Access API

In [3]:

# Authenticate to Twitter
auth = tweepy.OAuthHandler("CkhGa0z1CKMlK7GY56Zxh0250", 
    "Iq1o1IwRzuFw6xCandQbGUX1gzx6GezPD7ek1ptbcw46eni3LB")
auth.set_access_token("492948056-n4H0mlG3efeuJnDKkkoyeW9BWbl7kBrI95Dmlup1", 
    "jMSNiTz1mo764Xnz1x8cEAuu2CQRIhRbstAgRiD5WY7cP")

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


# Important Function

## Koneksi Database

In [4]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

## Predict by model

In [5]:
URL = 'http://tf-serving-sentiment.herokuapp.com/v1/models/sentiment:predict'
def get_predict(series, URL_Serving):
    X=[]
    for con in series.tolist():
        X.append([con])

    input_data_json = json.dumps({
        "signature_name": "serving_default",
        "instances": X,
        })
        
    r=requests.post(URL_Serving, input_data_json)
    result=r.json()
    result= np.argmax(result['predictions'], axis=1)
    return np.where(result==0,'Negative', np.where(result==1, 'Neutral', 'Positive'))

## Search tweet

In [32]:
def get_tweet(query, api, cursor, result_type='recent'):
    max_id=cursor.execute('SELECT MAX(ID) FROM tweet').fetchall()[0][0]
    data = api.search_tweets(q=query, 
                        result_type=result_type,
                        count=100, 
                        tweet_mode ='extended', 
                        since_id= max_id)
    data= pd.DataFrame([[s.id, s.full_text.replace('\n','').replace('\r',''), s.user.screen_name, s.user.id_str, s.user.followers_count, s.retweet_count, s.favorite_count, s.created_at] for s in data], columns=('ID', 'Texts', 'UserName','UserID', "UserFollowerCount", 'RetweetCount', 'Likes', "CreatedAt"))
    if data.shape[0]>0: 
        data['sentimen']=get_predict(data['Texts'], URL)
        for id in data[(data.sentimen=="Negative")&(data.UserID!='1476341100260388867')].ID:
            try:
                status="Halo kak @"+data[data.ID==id]["UserName"][0]+". maaf atas ketidaknyamanannya. Untuk informasi lebih lanjut, cek dm ya"
                api.update_status(status, in_reply_to_status_id=id)
                api.send_direct_message(data[data.ID==id]["UserID"][0],"Maaf kak atas kendala yang dihadapi. Bisa diceritakan lebih detail terkait masalah yang kakak hadapi?")
            except Exception as error:
                print(error)
    return data
    # else: return "tidak ada tweet"


# Main

## Connect Database

In [7]:
con = create_connection("twit copy.db")
cursor=con.cursor()

## Cek limit request

In [8]:
api.rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 175, 'reset': 1659178928}}

## Run Constant

In [ ]:
while True:
    new_data=get_tweet('"bank t0yip" OR t0yip',api, cursor)
    new_data.to_sql('tweet', con=con, method='multi', if_exists='append', index=False)
    print(f'Data baru sejumlah {new_data.shape[0]}')
    sleep(20)